# Module 1: Agent Foundations
*Building Your First Autonomous AI Agent*

**Learning Objectives:**
- Understand core agent components
- Implement the ReAct pattern
- Build tools and memory systems

**Duration:** 45 minutes

In [ ]:
import openai
import json
from typing import Dict, Any, List
from dataclasses import dataclass
from enum import Enum
import os

print("Environment setup complete")

## Agent Architecture

Every agent has four components:
1. Brain (LLM)
2. Memory
3. Tools
4. Planning

In [ ]:
class AgentState(Enum):
    IDLE = "idle"
    THINKING = "thinking"
    ACTING = "acting"
    WAITING = "waiting"
    ERROR = "error"

@dataclass
class AgentAction:
    tool_name: str
    tool_input: str
    reasoning: str

@dataclass
class BasicAgentConfig:
    name: str = "Agent"
    max_iterations: int = 5
    verbose: bool = True
    enable_planning: bool = False
    timeout_seconds: int = 30

class BasicAgent:
    def __init__(self, name: str):
        self.name = name
        self.tools = {}
        self.memory = []
        self.state = AgentState.IDLE
        self.current_task = None
        self.simple_context = []
        self.iteration_count = 0
        self.config = BasicAgentConfig(name=name)
    
    def add_tool(self, name: str, func, desc: str):
        self.tools[name] = {'function': func, 'description': desc}
        print(f"Added tool: {name}")
    
    def update_state(self, new_state: AgentState):
        print(f"State change: {self.state.value} → {new_state.value}")
        self.state = new_state

agent = BasicAgent("ResearchBot")
print(f"Created agent: {agent.name} (State: {agent.state.value})")

## Building Tools

Tools extend agent capabilities beyond text generation.

In [ ]:
def safe_tool_execution(tool_func, tool_input: str):
    """Foundation-level error handling"""
    try:
        result = tool_func(tool_input)
        return {"success": True, "result": result, "error": None}
    except Exception as e:
        return {"success": False, "result": None, "error": str(e)}

def handle_tool_failure(agent, error_msg: str):
    """Simple error recovery strategies"""
    print(f"❌ Tool failed: {error_msg}")
    print("🔄 Trying alternative approach...")
    
    fallback_options = [
        "Try a different tool",
        "Simplify the request", 
        "Ask for clarification",
        "Use default response"
    ]
    
    chosen_fallback = fallback_options[0]
    print(f"📋 Fallback strategy: {chosen_fallback}")
    return chosen_fallback

def web_search(query: str) -> str:
    """Simulated web search"""
    results = {
        "ai": "Artificial Intelligence transforms industries",
        "climate": "Climate change affects global weather patterns",
        "agent": "AI agents are autonomous software systems"
    }
    for key, result in results.items():
        if key in query.lower():
            return f"Found: {result}"
    return "General search results found"

def calculator(expr: str) -> str:
    """Safe calculator with error handling"""
    try:
        if all(c in '0123456789+-*/().' for c in expr):
            return str(eval(expr))
        return "Invalid expression"
    except:
        return "Calculation error"

agent.add_tool("search", web_search, "Search the web")
agent.add_tool("calc", calculator, "Calculate math expressions")

print("🧪 Testing error handling:")
safe_result = safe_tool_execution(agent.tools["calc"]["function"], "invalid")
print(f"Safe execution result: {safe_result}")

print(f"Agent has {len(agent.tools)} tools")

## Enhanced ReAct Pattern

The ReAct pattern (Reasoning + Acting) is the foundation of agent behavior:

1. **Think** about the goal and current situation
2. **Act** by choosing and executing an action  
3. **Observe** the results
4. **Repeat** until goal achieved

### Visual Diagram

In [ ]:
def show_react_pattern_diagram():
    """Visual explanation of ReAct pattern"""
    print("""
    🧠 ReAct Pattern (Reasoning + Acting):
    
    ┌─────────────┐
    │   THINK     │ ← Analyze situation
    └──────┬──────┘
           ↓
    ┌─────────────┐
    │    ACT      │ ← Choose and execute action  
    └──────┬──────┘
           ↓
    ┌─────────────┐
    │  OBSERVE    │ ← Process results
    └──────┬──────┘
           ↓
    ┌─────────────┐
    │   REPEAT    │ ← Until goal achieved
    └─────────────┘
    """)

def think_about_task(goal: str, context: List[str]) -> str:
    """Generate reasoning for current task"""
    if "search" in goal.lower():
        return f"I need to find information about '{goal}'. Let me search for relevant data."
    elif "calculate" in goal.lower():
        return f"This requires mathematical computation. I'll use the calculator tool."
    elif "analyze" in goal.lower():
        return f"I need to break down and examine '{goal}' systematically."
    else:
        return f"I need to understand what '{goal}' requires and choose appropriate tools."

def take_action(thought: str, tools: Dict) -> str:
    """Execute action based on reasoning"""
    if "search" in thought.lower():
        return tools["search"]["function"]("relevant query")
    elif "calculat" in thought.lower():
        return tools["calc"]["function"]("2+2")
    else:
        return "Action completed"

def observe_result(action_result: str) -> str:
    """Process and interpret action results"""
    if "Found:" in action_result:
        return f"Successfully retrieved information: {action_result}"
    elif action_result.isdigit():
        return f"Calculation completed with result: {action_result}"
    else:
        return f"Action completed: {action_result}"

def complete_react_demo(agent, goal: str, max_iterations: int = 3):
    """Complete ReAct cycle for foundations"""
    print(f"🎯 GOAL: {goal}")
    agent.update_state(AgentState.THINKING)
    
    for i in range(max_iterations):
        print(f"\n--- Cycle {i+1} ---")
        
        # THINK
        print("💭 THINKING...")
        thought = think_about_task(goal, agent.simple_context)
        print(f"   {thought}")
        
        # ACT  
        print("🔧 ACTING...")
        agent.update_state(AgentState.ACTING)
        action_result = take_action(thought, agent.tools)
        print(f"   {action_result}")
        
        # OBSERVE
        print("👁️  OBSERVING...")
        agent.update_state(AgentState.WAITING)
        observation = observe_result(action_result)
        print(f"   {observation}")
        
        agent.simple_context.append(f"{thought} → {observation}")
        
        if "complete" in observation.lower() or "successfully" in observation.lower():
            print("✅ Goal achieved!")
            agent.update_state(AgentState.IDLE)
            return True
    
    print("⏰ Reached iteration limit")
    agent.update_state(AgentState.IDLE)
    return False

show_react_pattern_diagram()

print("\n🧪 Enhanced ReAct Demo:")
complete_react_demo(agent, "search for AI agent information")

## Basic Planning & Communication

Simple planning and communication capabilities for foundation-level agents.

In [ ]:
class SimpleGoalPlanner:
    """Basic planning for Module 1"""
    
    def break_down_goal(self, goal: str) -> List[str]:
        """Simple goal decomposition"""
        print(f"🎯 Breaking down goal: '{goal}'")
        
        if "search" in goal.lower():
            return ["identify search terms", "execute search", "review results"]
        elif "calculate" in goal.lower():
            return ["parse numbers", "choose operation", "compute result"]
        elif "analyze" in goal.lower():
            return ["gather data", "process information", "draw conclusions"]
        else:
            return ["understand request", "gather resources", "execute task"]
    
    def create_action_sequence(self, subgoals: List[str]) -> List[Dict]:
        """Convert subgoals to actionable steps"""
        actions = []
        for i, subgoal in enumerate(subgoals, 1):
            actions.append({
                "step": i,
                "action": subgoal,
                "status": "pending",
                "tool_needed": self.determine_tool_needed(subgoal)
            })
        return actions
    
    def determine_tool_needed(self, subgoal: str) -> str:
        """Simple tool selection logic"""
        if "search" in subgoal.lower():
            return "search"
        elif "calculat" in subgoal.lower() or "compute" in subgoal.lower():
            return "calc"
        else:
            return "general"

def agent_to_human_communication(agent, message: str):
    """Foundation-level communication patterns"""
    communication_templates = {
        "ask_clarification": "I need more information about: {topic}",
        "report_progress": "I have completed: {action}. Next: {next_action}",
        "report_error": "I encountered an issue: {error}. Suggested solution: {solution}",
        "confirm_action": "Should I proceed with: {action}?"
    }
    
    if "?" in message:
        return communication_templates["ask_clarification"].format(topic=message)
    else:
        return communication_templates["report_progress"].format(
            action=message, 
            next_action="awaiting instructions"
        )

# Demo planning system
planner = SimpleGoalPlanner()
goal = "search and analyze AI agent information"
subgoals = planner.break_down_goal(goal)
action_sequence = planner.create_action_sequence(subgoals)

print("🗓️  Planning Demo:")
print(f"Goal: {goal}")
print(f"Subgoals: {subgoals}")
print(f"Action Sequence: {action_sequence}")

print("\n💬 Communication Demo:")
comm1 = agent_to_human_communication(agent, "What should I search for?")
comm2 = agent_to_human_communication(agent, "completed search task")
print(f"Question response: {comm1}")
print(f"Progress report: {comm2}")

## Hands-On Exercises

Practice building and debugging agents with these interactive exercises.

In [ ]:
def exercise_1_build_basic_agent():
    """Hands-on: Build your first complete agent"""
    print("🛠️  EXERCISE 1: Build a Calculator Agent")
    print("Goal: Create an agent that can solve math problems step by step")
    print()
    print("📋 Template provided below - students complete the implementation:")
    
    template_agent = BasicAgent("MathBot")
    
    def advanced_calculator(expression: str) -> str:
        print(f"TODO: Implement advanced calculation for: {expression}")
        return "Student implementation needed"
    
    def explain_calculation_steps(expression: str) -> List[str]:
        print(f"TODO: Explain steps for: {expression}")
        return ["Step 1: TODO", "Step 2: TODO"]
    
    template_agent.add_tool("advanced_calc", advanced_calculator, "Advanced calculator with explanations")
    
    print("🎯 Your tasks:")
    print("1. Complete the advanced_calculator function")
    print("2. Implement step-by-step reasoning")
    print("3. Add comprehensive error handling")
    print("4. Test with complex expressions like '(2+3)*4-1'")
    
    return template_agent

def exercise_2_debug_broken_agent():
    """Hands-on: Debug this intentionally broken agent"""
    print("🐛 EXERCISE 2: Debug the Broken Agent")
    print("Find and fix the bugs in this agent!")
    print()
    
    broken_code = '''
def broken_search_agent(query):
    # Bug 1: tool doesn't exist
    tools = {"search": broken_search_tool}  
    
    # Bug 2: no error handling
    result = tools["search"](query)
    
    # Bug 3: assuming result is always a string
    return result.upper()  # What if result is None?

def broken_search_tool(query):
    # Bug 4: inconsistent return types
    if query == "test":
        return None  # Sometimes returns None
    elif len(query) < 3:
        raise ValueError("Query too short")  # Sometimes raises exception
    else:
        return {"data": "some result"}  # Sometimes returns dict
    '''
    
    print("🔍 Broken Code:")
    print(broken_code)
    print()
    print("🎯 Find these bugs:")
    print("1. Non-existent tool reference")
    print("2. Missing error handling") 
    print("3. Unsafe string operations")
    print("4. Inconsistent return types")

print("🎓 FOUNDATION EXERCISES")
print("=" * 50)

exercise_1_build_basic_agent()
print("\n" + "="*50)

exercise_2_debug_broken_agent()
print("\n" + "="*50)

print("\n✅ Complete these exercises to master agent foundations!")
print("💡 Solutions and advanced patterns in Module 2: Memory & Learning")

## Module 1 Summary

🎯 **What You Built:**

**Core Agent Components:**
- ✅ Agent state management and lifecycle
- ✅ Basic configuration system  
- ✅ Tool integration with error handling
- ✅ Enhanced ReAct reasoning pattern

**Foundation Skills:**
- ✅ Goal decomposition and basic planning
- ✅ Agent-human communication patterns
- ✅ Error recovery strategies
- ✅ Visual flow understanding

**Hands-On Practice:**
- ✅ Calculator agent implementation
- ✅ Debugging broken agent code
- ✅ Step-by-step guided exercises

**Key Patterns Learned:**
1. **ReAct Cycle**: Think → Act → Observe → Repeat
2. **State Management**: Track agent status throughout execution
3. **Error Handling**: Safe tool execution with fallback strategies
4. **Basic Planning**: Simple goal decomposition techniques

**What's Next:**
- **Module 2**: Memory & Learning - Persistent storage, experience replay, performance optimization
- **Module 3**: Tool Integration - Database connections, APIs, file processing  
- **Module 4**: Advanced Planning - HTN algorithms, goal hierarchies, adaptive execution

🏗️ **Foundation Complete!** You now understand core agent architecture and can build basic autonomous systems.